In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import the libraries as shown below

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore')
from tensorflow import  keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import  categorical_crossentropy
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import  imagenet_utils
from sklearn.metrics import  confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt

In [ ]:
datasets,info=tfds.load(name='plant_village',with_info=True,as_supervised=True,split=['train'])

In [ ]:
info

In [ ]:
train,info_train=tfds.load(name='plant_village',with_info=True,split='train')
tfds.show_examples(info_train,train)

In [ ]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image /=255.0

  return tf.image.resize(image,[224,224]),tf.one_hot(label,38) 

In [ ]:
def get_dataset(batch_size=32):
  train_dataset_scaled=datasets[0].map(scale).shuffle(1000).batch(batch_size)
  
  return train_dataset_scaled

In [ ]:
train_dataset=get_dataset()
train_dataset.cache()


In [ ]:
len(list(datasets[0]))

In [ ]:
feature_extractor="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
feature_extractor_layer=hub.KerasLayer(feature_extractor,input_shape=(224,224,3))

In [ ]:
feature_extractor_layer.trainable=False

In [ ]:
model=tf.keras.Sequential([
                           feature_extractor_layer,
                           tf.keras.layers.Dropout(0.3),
                           tf.keras.layers.Dense(38,activation='softmax')

                           

])
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['acc']

)

In [ ]:
history=model.fit(train_dataset,epochs=10)

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/Leaf_Disease/jupyterNote book/PlantVillage_M1.h5')


In [ ]:
result=model.evaluate(train_dataset)

In [ ]:
datasets[0].take(100)

In [ ]:
for test_sample in datasets[0].take(10):
  image,label=test_sample[0],test_sample[1]
  image_scaled,label_arr=scale(test_sample[0],test_sample[1])
  image_scaled=np.expand_dims(image_scaled,axis=0)

  img=tf.keras.preprocessing.image.img_to_array(image)
  pred=model.predict(image_scaled)
  print(pred)
  plt.figure()
  plt.imshow(image)
  plt.show()

  print("Actual Label: %s" % info.features["label"].names[label.numpy()])
  print("Predicted Label: %s" % info.features["label"].names[np.argmax(pred)])

In [ ]:
for f0,f1 in datasets[0].map(scale).batch(200):
  y=np.argmax(f1,axis=1)
  y_pred=np.argmax(model.predict(f0),axis=1)
  print(tf.math.confusion_matrix(labels=y, predictions=y_pred, num_classes=38))